# Demo Feeder Concept
This notebook shows a basics of a feeder concept. This scenario has two vessels:
* a *vessel* for the transport of material
* an *installer*, receiving the material and installing it.
Thus, we also need two processes - one for each vessel. In this process the hand over of the material is done from the *vessel* to the *installer*.

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import pandas as pd
import numpy as np

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import uuid

from plot import vessel_planning

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

## Definition of Sites

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasMultiContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 5, "capacity": 10},
        {"id": "TP", "level": 5, "capacity": 10},
    ],
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 0, "capacity": 5},
        {"id": "TP", "level": 0, "capacity": 5},
    ],
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)


init
{'id': 'MP', 'level': 5, 'capacity': 10}
{'id': 'TP', 'level': 5, 'capacity': 10}
init
{'id': 'MP', 'level': 0, 'capacity': 5}
{'id': 'TP', 'level': 0, 'capacity': 5}


## Definition of Vessels

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.MultiContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
        # SiteRegistry,
    ),
    {"key": "MultiStoreHopper"},
)

# print(SiteRegistry.inspect("MultiStoreHopper"))
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10


# TSHD variables
data_vessel = {
    "env": my_env,  # The simpy environment
    "name": "Vessel 01",  # Name
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 0, "capacity": 2},
        {"id": "TP", "level": 0, "capacity": 2},
    ],  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
vessel = TransportProcessingResource(**data_vessel)


data_installer = {
    "env": my_env,  # The simpy environment
    "name": "Installer",  # Name
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 0, "capacity": 1},
        {"id": "TP", "level": 0, "capacity": 1},
    ],  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
installer = TransportProcessingResource(**data_installer)

init
{'id': 'MP', 'level': 0, 'capacity': 2}
{'id': 'TP', 'level': 0, 'capacity': 2}
init
{'id': 'MP', 'level': 0, 'capacity': 1}
{'id': 'TP', 'level': 0, 'capacity': 1}


## Definition of Vessel process

In [4]:
# definition of vessel main cycle
def vessel_process(vessel):
    single_run = []
    move_activity_to_harbor_data = {
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "sailing empty",  # We are moving soil
        "ID": str(uuid.uuid4()),  # For logging purposes
        "registry": registry,
        "mover": vessel,
        "destination": from_site,
        "postpone_start": True,
    }
    single_run.append(model.MoveActivity(**move_activity_to_harbor_data))
    
    shift_amount_loading_data = {
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "Load MP",  # We are moving soil
        "ID": str(uuid.uuid4()),  # For logging purposes
        "registry": registry,
        "processor": vessel,
        "origin": from_site,
        "destination": vessel,
        "amount": 1,
        "duration": 120,
        "id_": "MP",
        "postpone_start": True,
    }
    single_run.append(model.ShiftAmountActivity(**shift_amount_loading_data))
    
    move_activity_to_site_data = {
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "sailing filled",  # We are moving soil
        "ID": str(uuid.uuid4()),  # For logging purposes
        "registry": registry,
        "mover": vessel,
        "destination": to_site,
        "postpone_start": True,
    }
    single_run.append(model.MoveActivity(**move_activity_to_site_data))
    
    shift_amount_unloading_data = {
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "Transfer MP",  # We are moving soil
        "ID": str(uuid.uuid4()),  # For logging purposes
        "registry": registry,
        "processor": installer,
        "origin": vessel,
        "destination": installer,
        "amount": 1,
        "duration": 120,
        "id_": "MP",
        "postpone_start": True,
        "start_event": [{"type":"activity", "name": "Sail transit" , "state":"done"}],
    }
    single_run.append(model.ShiftAmountActivity(**shift_amount_unloading_data))
    
    sequential_activity_data3 = {
        "env": my_env,
        "name": "Single run process",
        "ID": str(uuid.uuid4()),  # For logging purposes
        "registry": registry,
        "sub_processes": single_run,
        "postpone_start": True,
    }
    activity = model.SequentialActivity(**sequential_activity_data3)
    
    while_data = {
        "env": my_env,  # The simpy environment defined in the first cel
        "name": "single run while",  # We are moving soil
        "ID": str(uuid.uuid4()),  # For logging purposes
        "registry": registry,
        "sub_process": activity,
        "condition_event": [{"type":"container", "concept": from_site, "state":"empty", "id_":"MP"}],
        "postpone_start": False,
    }
    while_activity = model.WhileActivity(**while_data)
    return while_activity


## Definition of Installer process

In [5]:
# installer process
#
installer_run = []
move_activity_to_site_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing empty",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "mover": installer,
    "destination": to_site,
    "postpone_start": True,
}
installer_run.append(model.MoveActivity(**move_activity_to_site_data))

installer_run2 = []
basic_activity_data= {"env"  : my_env,
                      "name" : "Sail transit",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 30,
                      "additional_logs": [installer],
                      "postpone_start": True,
                      }
installer_run2.append(model.BasicActivity(**basic_activity_data))

basic_activity_data33= {"env"  : my_env,
                      "name" : "Prepare installation",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 60,
                      "additional_logs": [installer],
                      "postpone_start": True,
                      "start_event": [{"type":"activity", "name": "Single run process" , "state":"done"}],
                      }
installer_run2.append(model.BasicActivity(**basic_activity_data33))

shift_amount_unloading_data2 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Install MP",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "processor": installer,
    "origin": installer,
    "destination": to_site,
    "amount": 1,
    "duration": 120,
    "id_": "MP",
    "postpone_start": True,
}
installer_run2.append(model.ShiftAmountActivity(**shift_amount_unloading_data2))

sequential_activity_data3 = {
    "env": my_env,
    "name": "Installer run process 2",
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_processes": installer_run2,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data3)

while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "installer run while",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": to_site, "state":"full", "id_":"MP"}],
    "postpone_start": True,
}
installer_run.append(model.WhileActivity(**while_data))

sequential_activity_data4 = {
    "env": my_env,
    "name": "Installer run process",
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_processes": installer_run,
    "postpone_start": False,
}
installer_proc = model.SequentialActivity(**sequential_activity_data4)

## instantiate vessel process
vessel_proc = vessel_process(vessel)


while Activity keep_resources []
start event instance None
while Activity keep_resources []
start put_available
register new event 
start event instance None


In [6]:
my_env.run()


keep_resources []
start event instance None
Mover_move before mover resource request
conditional 
start event instance None
keep_resources []
start event instance None
Mover_move before mover resource request
Mover_move after mover resource request
Mover_move after mover resource request
Mover_move after move
Activity end()
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.00a6a6b1-88ab-4703-a3e8-c2333b1f214c': 5, 'destination.dad123f9-d288-4b50-883a-aa9770043fbc': 2}
destination request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de8024e08>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de8024e08>}
site request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de8024e08>, <simpy.resources.resource.Resource object at 

Mover_move after move
Activity end()
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.00a6a6b1-88ab-4703-a3e8-c2333b1f214c': 3, 'destination.dad123f9-d288-4b50-883a-aa9770043fbc': 2}
destination request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93cfc48>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93cfc48>}
site request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93cfc48>, <simpy.resources.resource.Resource object at 0x0000022DE93BB2C8>: <Request() object at 0x22de93c98c8>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93cfc48>, <simpy.resources.resource.Resource object at 0x0000022DE93BB2C8>: <Request() object at 0x22de93c98c8>}

Activity end()
condition event triggered: False False round 2
while loop requested_resources {}
while loop keep_resources []
conditional 
start event instance None
keep_resources []
start event instance None
Activity end()
keep_resources []
start event expression [{'type': 'activity', 'name': 'Single run process', 'state': 'done'}]
start event <Event() object at 0x22de93e6a88>
start event instance <Event() object at 0x22de93e6a88>
delayed process : <Event() object at 0x22de93e6a88>
Mover_move after move
Activity end()
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.00a6a6b1-88ab-4703-a3e8-c2333b1f214c': 2, 'destination.dad123f9-d288-4b50-883a-aa9770043fbc': 2}
destination request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93e4648>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Re

keep_resources []
start event expression [{'type': 'activity', 'name': 'Single run process', 'state': 'done'}]
start event <Event() object at 0x22de93d6cc8>
start event instance <Event() object at 0x22de93d6cc8>
delayed process : <Event() object at 0x22de93d6cc8>
Mover_move after move
Activity end()
keep_resources []
SHift amount Activity keep_resources []
start event instance None
start : {}
{'origin.00a6a6b1-88ab-4703-a3e8-c2333b1f214c': 1, 'destination.dad123f9-d288-4b50-883a-aa9770043fbc': 2}
destination request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93d33c8>}
shift amount process keep_resources []
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93d33c8>}
site request : {<simpy.resources.resource.Resource object at 0x0000022DE93CF1C8>: <Request() object at 0x22de93d33c8>, <simpy.resources.resource.Resource object at 0x0000022DE93BB2C8>: <Req

In [7]:
log_df = pd.DataFrame(vessel.log)
data = log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data = data.drop_duplicates()
data


,Message,ActivityState,Timestamp,Value,ActivityID
0,sailing empty,START,1970-01-01 01:00:00.000000,0,4b6be67e-64b5-4353-9435-36463f4d2e17
1,sailing empty,STOP,1970-01-01 01:00:00.000000,0,4b6be67e-64b5-4353-9435-36463f4d2e17
2,transfer MP to Vessel 01,START,1970-01-01 01:00:00.000000,1,7de2c9b8-a417-4ab4-ace1-96d4a23a7f07
4,transfer MP to Vessel 01,STOP,1970-01-01 01:02:00.000000,1,7de2c9b8-a417-4ab4-ace1-96d4a23a7f07
6,sailing filled,START,1970-01-01 01:02:00.000000,0,9fd375bc-864c-4d16-86ef-c9a3728d8af5
7,sailing filled,STOP,1970-01-01 01:17:42.824591,0,9fd375bc-864c-4d16-86ef-c9a3728d8af5
8,transfer MP to Installer,START,1970-01-01 01:17:42.824591,1,5f7b418e-f573-4b0e-8640-f76b18c484d4
9,transfer MP to Installer,STOP,1970-01-01 01:19:42.824591,1,5f7b418e-f573-4b0e-8640-f76b18c484d4
10,sailing empty,START,1970-01-01 01:19:42.824591,0,4b6be67e-64b5-4353-9435-36463f4d2e17
11,sailing empty,STOP,1970-01-01 01:35:25.649183,0,4b6be67e-64b5-4353-9435-36463f4d2e17


In [8]:
log_df = pd.DataFrame(installer.log)
data2 = log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data2 = data2.drop_duplicates()
data2

,Message,ActivityState,Timestamp,Value,ActivityID
0,sailing empty,START,1970-01-01 01:00:00.000000,0,88bc7a42-3018-48b3-8acf-350afc5b1bd4
1,sailing empty,STOP,1970-01-01 01:15:42.824591,0,88bc7a42-3018-48b3-8acf-350afc5b1bd4
2,Sail transit,START,1970-01-01 01:15:42.824591,30,e5523cd5-5f91-4a5e-9ac0-4d44ac1d2484
3,Sail transit,STOP,1970-01-01 01:16:12.824591,30,e5523cd5-5f91-4a5e-9ac0-4d44ac1d2484
4,Prepare installation,WAIT_START,1970-01-01 01:16:12.824591,-1,c2642439-5f3a-41cf-bee6-2f556df272ef
5,transfer MP to Installer,START,1970-01-01 01:17:42.824591,1,5f7b418e-f573-4b0e-8640-f76b18c484d4
7,transfer MP to Installer,STOP,1970-01-01 01:19:42.824591,1,5f7b418e-f573-4b0e-8640-f76b18c484d4
9,Prepare installation,WAIT_STOP,1970-01-01 01:19:42.824591,-1,c2642439-5f3a-41cf-bee6-2f556df272ef
10,Prepare installation,START,1970-01-01 01:19:42.824591,60,c2642439-5f3a-41cf-bee6-2f556df272ef
11,Prepare installation,STOP,1970-01-01 01:20:42.824591,60,c2642439-5f3a-41cf-bee6-2f556df272ef


In [9]:
objects = [vessel, installer]
activities = []
for obj in objects:
    print(set(obj.log["Message"]))
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

            
C = np.linspace(0,255, len(activities))
colors = {}

for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

vessel_planning(objects, activities, colors)

{'transfer MP to Vessel 01', 'sailing filled', 'sailing empty', 'transfer MP to Installer'}
{'sailing empty', 'Prepare installation', 'transfer MP to Installer', 'Sail transit', 'transfer MP to Dumplocatie'}
